# Correlating NYT Data with Market Data

### Loading in Data

In [1]:
import os
import json
import time
import requests
import datetime
import dateutil
import pandas as pd
import glob
import string
from dateutil.relativedelta import relativedelta
import numpy as np
import yfinance as yf

In [2]:
spy = yf.download("SPY")
nyt = pd.read_csv("./Data/NYTDataCleaned.csv", encoding = "ISO-8859-1")

xlk = yf.download("XLK")
xlv = yf.download("XLV")
xlf = yf.download("XLF")
xly = yf.download("XLY")
xli = yf.download("XLI")
xlp = yf.download("XLP")
xlu = yf.download("XLU")
xle = yf.download("XLE")
xlc = yf.download("XLC")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [11]:
def modifyDataSet(fund):
    return np.log(fund["Open"]-fund["Close"]).dropna()
    
listOfFunds = [spy , xlk , xlv, xlf, xly , xli, xlp, xlu, xle, xlc]
for etf in listOfFunds

SyntaxError: invalid syntax (<ipython-input-11-0f65a1279def>, line 5)

In [ ]:
xlc["Log Returns"]

In [4]:
# Match nyt data to the data we have on SPY and combine them into one dataframe
# spy = spy.iloc[6969:] # Remove this time
spy.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-01-29,43.96875,43.96875,43.75000,43.93750,26.079659,1003200
1993-02-01,43.96875,44.25000,43.96875,44.25000,26.265144,480500
1993-02-02,44.21875,44.37500,44.12500,44.34375,26.320782,201300
1993-02-03,44.40625,44.84375,44.37500,44.81250,26.599014,529400
1993-02-04,44.96875,45.09375,44.46875,45.00000,26.710312,531500
